#Criação do dateset de estados quânticos preparados por método

Este repositório foi feito durante no Pesquisa do Trabalho de Conclusão de Curso referente a Graduação de Sistema de Informação, o artigo se encontra neste link:

Estados utilizados: Uniforme Denso, Uniforme Esparso (25%, 50%,75%), Não-Uniforme Denso, Não-Uniforme Esparso (25%, 50%,75%);

Métodos utilizados: Low Rank,  Dcsp, Bdsp, SVD, UCG, Isometry e Multiplexor


In [1]:
%%capture
!pip install -U qiskit
!pip install -U qiskit-aer
!pip install qclib

In [2]:
import numpy as np
import pandas as pd
import random as rnd, random
import string
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import AerSimulator
from qclib.state_preparation import (
    LowRankInitialize,
    DcspInitialize,
    BdspInitialize,
    SVDInitialize,
    UCGInitialize)

from google.colab import files
from datetime import datetime
import time

##Funções do controle do Dataframe

In [3]:
cabecalho = ['Identifier','State', 'Qubit', 'Uniform State', 'Dense State', 'Sparsity','Duration','Output Qubit','Output CNot Gate','Output Width', 'Output Depth', 'Preparation Method']

df = pd.DataFrame(columns=cabecalho)

df.head()

data = {
    'Identifier': [],
    'State': [],
    'Qubit': [],
    'Uniform State': [],
    'Dense State': [],
    'Sparsity': [],
    'Duration': [],
    'Output Qubit': [],
    'Output CNot Gate': [],
    'Output Depth': [],
    'Output Width': [],
    'Preparation Method': []
}

def restartData(data): #Essa reseta/limapa do dicionario
  data = {
      'Identifier': [],
      'State': [],
      'Qubit': [],
      'Uniform State': [],
      'Dense State': [],
      'Sparsity': [],
      'Duration': [],
      'Output Qubit': [],
      'Output CNot Gate': [],
      'Output Depth': [],
      'Output Width': [],
      'Preparation Method': []
    }
  return data

def updateData(id,data, state, num_qubits, uniform, dense, sparsity, duration, op_qubit, op_cnot, op_depth, op_width, preparation_method): #Essa funcao atualiza o dicionario com novos dados
    novos_dados = {
        'Identifier': id,
        'State': state,
        'Qubit': num_qubits,
        'Uniform State': int(uniform),
        'Dense State': int(dense),
        'Sparsity': sparsity,
        'Duration': duration,
        'Output Qubit': op_qubit,
        'Output CNot Gate': op_cnot,
        'Output Depth': op_depth,
        'Output Width': op_width,
        'Preparation Method': preparation_method
    }

    for chave, valor in novos_dados.items():
        data[chave].append(valor)
    return data

def addDF(data): #Essa funcao adiciona as caracteristicas do estado + duracao + o resultado dado o método em um tabela/dataframe
    global df
    new_data = data
    df = pd.concat([df, pd.DataFrame(new_data)], ignore_index=False)
    return df


def getDuration(before_time, after_time):
    duration = after_time - before_time
    microseconds = duration.total_seconds() * 1000000
    return microseconds


##Geração dos Estados Quânticos

In [4]:
def denseUniformState(num_qubits):
  state_vector= np.ones(2 ** num_qubits)
  state_vector = state_vector/np.linalg.norm(state_vector)
  uniform = True
  dense = True

  return [state_vector, num_qubits,uniform, dense, 0]

def sparseUniformState(num_qubits, sparsity):
  s = int((sparsity)*num_qubits**2)
  intindex=[]
  for i in range(num_qubits**2-1):
    intindex.append(i)
  zero_index =rnd.sample(intindex,s)
  state_vector = np.ones(2 ** num_qubits)
  for i in zero_index:
    state_vector[i]=0.
  state_vector = state_vector/np.linalg.norm(state_vector)
  uniform = True
  dense = False
  return [state_vector,num_qubits, uniform, dense, sparsity]

def denseNonUniformState(num_qubits):
  state_vector= np.random.rand(2 ** num_qubits)
  state_vector = state_vector/np.linalg.norm(state_vector)
  uniform = False
  dense = True
  return [state_vector,num_qubits, uniform, dense, 0]

def sparseNonUniformState(num_qubits, sparsity):
  s = int((sparsity)*num_qubits**2)
  intindex=[]
  for i in range(num_qubits**2-1):
    intindex.append(i)
  zero_index =rnd.sample(intindex,s)
  state_vector= np.random.rand(2 ** num_qubits)
  for i in zero_index:
    state_vector[i]=0.
  state_vector = state_vector/np.linalg.norm(state_vector)
  uniform = False
  dense = False
  return [state_vector,num_qubits, uniform, dense, sparsity]

##Funções dos métodos de preparação de estados quânticos

In [5]:
def lowrank(l,id):
  global data
  v = l[0]
  c = QuantumCircuit(l[1])
  before_time = datetime.now()
  LowRankInitialize.initialize(c, v)
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)
  after_time = datetime.now()
  duration = getDuration(before_time, after_time)

  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'Low Rank'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)

def svd(l,id):
  global data
  v = l[0]
  c = QuantumCircuit(l[1])
  before_time = datetime.now()
  SVDInitialize.initialize(c, v)
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)
  after_time = datetime.now()
  duration = getDuration(before_time, after_time)

  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'SVD'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)

def ucg(l,id):
  global data
  v = l[0]
  c = QuantumCircuit(l[1])
  before_time = datetime.now()
  UCGInitialize.initialize(c, v)
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)
  after_time = datetime.now()
  duration = getDuration(before_time, after_time)

  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'UCG'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)


def bdsp(l,id):
  global data
  v = l[0]
  before_time = datetime.now()
  c = BdspInitialize(v).definition
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)
  after_time = datetime.now()
  duration = getDuration(before_time, after_time)
  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'BDSP'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)

def dcsp(l,id):
  global data
  v = l[0]
  before_time = datetime.now()
  c = DcspInitialize(v).definition
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)
  after_time = datetime.now()
  duration = getDuration(before_time, after_time)
  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'DCSP'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)

def multiplexor(l,id):
  global data
  v = l[0]
  c = QuantumCircuit(l[1])
  before_time = datetime.now()
  c.initialize(v)
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)
  after_time = datetime.now()
  duration = getDuration(before_time, after_time)
  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'Multiplexor'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)


def isometry(l,id):
  global data
  v = l[0]
  before_time = datetime.now()
  r = QuantumRegister(l[1])
  c = QuantumCircuit(r)
  c.isometry(v, q_input=[], q_ancillas_for_output=r)
  transpiled = transpile(c, basis_gates=['u', 'cx'], optimization_level=0)

  after_time = datetime.now()
  duration = getDuration(before_time, after_time)
  op_qubit = len(transpiled.qubits)
  op_depth = transpiled.depth()
  op_width = transpiled.width()
  op_cnot = transpiled.count_ops().get('cx', 0)
  preparation_method  = 'Isometry'

  updateData(id=id,data=data, state=v, num_qubits=l[1], uniform=l[2], dense=l[3], sparsity=l[4],duration=duration,op_qubit=op_qubit, op_cnot=op_cnot, op_depth=op_depth, op_width=op_width, preparation_method=preparation_method)

##Geração do Dataset

Neste bloco de código, há funções da geração do ID junto com a lógica das funções que desenvolvem os estados e sua preparação, até incrementação no Dataframe.

In [8]:
def generateIdentifier(tamanho=30):
    caracteres = string.ascii_letters + string.digits
    palavra_aleatoria = ''.join(random.choice(caracteres) for _ in range(tamanho))
    return palavra_aleatoria

def addBd(state, id):
    try:
      lowrank(state, id)
    except Exception as e:
      print(f"Erro ao adicionar lowrank: {e}")
    try:
      svd(state, id)
    except Exception as e:
      print(f"Erro ao adicionar svd: {e}")
    try:
      ucg(state, id)
    except Exception as e:
      print(f"Erro ao adicionar ucg: {e}")
    try:
      bdsp(state, id)
    except Exception as e:
      print(f"Erro ao adicionar bdsp: {e}")
    try:
      dcsp(state, id)
    except Exception as e:
      print(f"Erro ao adicionar dcsp: {e}")
    try:
      multiplexor(state, id)
    except Exception as e:
      print(f"Erro ao adicionar multiplexor: {e}")
    try:
      isometry(state, id)
    except Exception as e:
      print(f"Erro ao adicionar isometry: {e}")

def createDenseUniformState(num_qubits):
    state = denseUniformState(num_qubits)
    id = generateIdentifier()
    addBd(state, id)

def createDenseNonUniformState(num_qubits):
    state = denseNonUniformState(num_qubits)
    id = generateIdentifier()
    addBd(state, id)

def createSparseUniformState(num_qubits, sparsity):
    state = sparseUniformState(num_qubits, sparsity)
    id = generateIdentifier()
    addBd(state, id)

def createSparseNonUniformState(num_qubits, sparsity):
    state = sparseNonUniformState(num_qubits, sparsity)
    id = generateIdentifier()
    addBd(state, id)


# Comentado cada preparação de estado quântico para criar csv separados.
def generateStates(nqubits):
  createDenseUniformState(nqubits)
  #createDenseNonUniformState(nqubits)

   #createSparseUniformState(nqubits,0.25)
   #createSparseUniformState(nqubits,0.5)
   #createSparseUniformState(nqubits,0.75)

   #createSparseNonUniformState(nqubits,0.25)
   #createSparseNonUniformState(nqubits,0.5)
   #createSparseNonUniformState(nqubits,0.75)


##Incrementador para função de geração de estados


In [ ]:
nome_arquivo = "DenseUniformState"
for _ in range(1,11):
  print(_)
  nQubits = 2
  while nQubits < 13:
    generateStates(nQubits)
    addDF(data)
    data = restartData(data)
    nQubits+=1

Salva o Dataset bruto em um arquivo csv

In [ ]:
df.to_csv(f'raw_{nome_arquivo}dataset.csv', index=False, sep=';')
files.download(f'raw_{nome_arquivo}dataset.csv')

##Tratamento do Dataset

O objetivo do tratamento é selecionar os melhores métodos por estado quântico com base no menor número de gates cnots no circuito e menor profundidade do circuito.

In [ ]:
# Supondo que você tenha um DataFrame chamado df
# com as colunas mencionadas ('State', 'Qubit', 'Uniform State', 'Dense State', 'Sparsity', 'Output CNot Gate', 'Output Depth')

# Ordenar o DataFrame pelos critérios em ordem
#df_ordenado = df.sort_values(by=['Output CNot Gate', 'Output Depth'], ascending=[True, True]):
#Esta linha ordena o DataFrame df com base nas colunas 'Output CNot Gate' e 'Output Depth'
#em ordem ascendente. O resultado é atribuído a um novo DataFrame chamado df_ordenado.
df_ordenado = df.sort_values(by=['Identifier','Output CNot Gate', 'Output Depth','Output Width'], ascending=[True, True, True, True])


# Converter a coluna problemática em uma representação hashável
#df_ordenado['State'] = df_ordenado['State'].apply(lambda x: tuple(x)): Aqui, a coluna 'State' do DataFrame df_ordenado é modificada.
#A função apply é usada para aplicar uma função a cada elemento da coluna 'State'. Neste caso, a função lambda é usada para converter
#cada valor da coluna 'State' em uma tupla. O resultado é atribuído de volta à coluna 'State' no DataFrame df_ordenado
#df_ordenado['Identifier'] = df_ordenado['Identifier'].apply(lambda x: tuple(x))

# Selecionar as melhores linhas
#melhores_linhas = df_ordenado.groupby(['State', 'Qubit', 'Uniform State', 'Dense State', 'Sparsity']).head(1): Aqui, o DataFrame df_ordenado
#é agrupado com base nas colunas 'State', 'Qubit', 'Uniform State', 'Dense State' e 'Sparsity'. A função groupby é utilizada para agrupar os dados
#com base nessas colunas. Em seguida, a função head(1) é aplicada para cada grupo, mantendo apenas a primeira linha de cada grupo.
#O resultado final é atribuído ao DataFrame melhores_linhas.

#melhores_linhas = df_ordenado.groupby(['State', 'Qubit', 'Uniform State', 'Dense State', 'Sparsity']).head(1)
melhores_linhas = df_ordenado.groupby(['Identifier']).head(1)
#melhores_linhas = melhores_linhas.drop('State_str', axis=1)
melhores_linhas = melhores_linhas.sort_values(by=['Qubit','Uniform State', 'Dense State', 'Sparsity'], ascending=[True,False, False, True])


# Exibir o DataFrame resultante com todas as colunas
#melhores_linhas.head(5): Finalmente, essa linha exibe as cinco primeiras linhas do DataFrame melhores_linhas, que contêm as primeiras linhas de cada grupo com base nas colunas mencionadas.
#melhores_linhas.head(5)

##Resumindo, o código ordena um DataFrame com base em duas colunas, converte uma coluna específica em tuplas, e depois mantém apenas a primeira linha de cada grupo com base em algumas colunas específicas. O resultado final é armazenado no DataFrame melhores_linhas e as cinco primeiras linhas desse DataFrame são exibidas.

#melhores_linhas.describe()
melhores_linhas.describe(include='all')

In [ ]:
melhores_linhas.to_csv(f'best_lines_{nome_arquivo}dataset.csv', index=False, sep=';')
files.download(f'best_lines_{nome_arquivo}dataset.csv')

In [ ]:
df_ordenado.to_csv(f'ordenano_{nome_arquivo}dataset.csv', index=False, sep=';')
files.download(f'ordenano_{nome_arquivo}dataset.csv')

In [ ]:
#melhores_linhas.head()